In [59]:
import sqlite3
import pandas as pd
import os
import glob
import re


In [60]:
csv_dir = "mobi_csv"
table_name = "rides"

In [ ]:
def standardize_column_string(name: str) -> str:
    # Remove any text within parentheses
    temp = re.sub(r'\([^()]*\)', '', name)
    temp = temp.strip().lower().replace(' ', '_')
    return temp

def standardize_column_names_df(df: pd.DataFrame) -> pd.DataFrame:
    standardized_columns = [standardize_column_string(col) for col in df.columns]
    df.columns = standardized_columns
    return df

In [ ]:
sample_csv = "mobi_csv/September_2025.csv"
sample_df = pd.read_csv(sample_csv, nrows=5)
sample_df = standardize_column_names_df(sample_df)
standardized_columns = sample_df.columns.tolist()
# standardized_columns
# ['departure',
#  'return',
#  'bike',
#  'electric_bike',
#  'departure_station',
#  'return_station',
#  'membership_type',
#  'covered_distance',
#  'duration',
#  'departure_temperature',
#  'return_temperature',
#  'stopover_duration',
#  'number_of_stopovers']




In [63]:
standardized_columns_datatypes = [
    'DATETIME',     # departure
    'DATETIME',     # return
    'INTEGER',      # bike
    'BOOLEAN',      # electric_bike
    'TEXT',         # departure_station
    'TEXT',         # return_station
    'TEXT',         # membership_type
    'INTEGER',      # covered_distance
    'INTEGER',      # duration
    'INTEGER',      # departure_temperature
    'INTEGER',      # return_temperature
    'INTEGER',      # stopover_duration
    'INTEGER'       # number_of_stopovers
]
standardized_columns_dict = dict(zip(standardized_columns, standardized_columns_datatypes))

In [64]:
conn = sqlite3.connect("mobi.db")
cursor = conn.cursor()

In [65]:
table_name = 'rides'
column_sql_strings = [f"{col_name} {col_type}" for col_name, col_type in standardized_columns_dict.items()]
create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_sql_strings)})"

In [66]:
column_sql_strings

['departure DATETIME',
 'return DATETIME',
 'bike INTEGER',
 'electric_bike BOOLEAN',
 'departure_station TEXT',
 'return_station TEXT',
 'membership_type TEXT',
 'covered_distance INTEGER',
 'duration INTEGER',
 'departure_temperature INTEGER',
 'return_temperature INTEGER',
 'stopover_duration INTEGER',
 'number_of_stopovers INTEGER']

In [67]:
cursor.execute(create_table_sql)
conn.commit()

In [68]:
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

In [69]:
print(f"Found {len(csv_files)} CSV files in directory '{csv_dir}':")                    

Found 92 CSV files in directory 'mobi_csv':


In [70]:
import pickle
finished_files = set()

if os.path.exists("finished_files.pkl"):
    with open("finished_files.pkl", "rb") as f:
        finished_files = pickle.load(f)



In [ ]:
def read_mobi_file(file):
    df = pd.read_csv(file)
    df = standardize_column_names_df(df)
    df = df[[x for x in df.columns if x in standardized_columns]]
    return df



In [72]:
import shutil

bad_files_dir = os.path.join(csv_dir, "bad_files")
os.makedirs(bad_files_dir, exist_ok=True)

for file in csv_files:
    if file not in finished_files:
        print(f"Importing file: {file}")
        try:
            df = read_mobi_file(file)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            finished_files.add(file)
        except:
            print(f"Failed to import file: {file}")
            shutil.move(file, os.path.join(bad_files_dir, os.path.basename(file)))
            finished_files.add(file)
        finally:
            pickle.dump(finished_files, open("finished_files.pkl", "wb"))
    else:
        print(f"Skipping already imported file: {file}")



Importing file: mobi_csv/May_2022.csv
Importing file: mobi_csv/July_2019.csv
Importing file: mobi_csv/January_2025.csv
Importing file: mobi_csv/January_2021.csv
Importing file: mobi_csv/June_2018.csv
Failed to import file: mobi_csv/June_2018.csv
Importing file: mobi_csv/July_2018.csv
Failed to import file: mobi_csv/July_2018.csv
Importing file: mobi_csv/August_2023.csv
Failed to import file: mobi_csv/August_2023.csv
Importing file: mobi_csv/March_2024.csv
Importing file: mobi_csv/February_2023.csv


/tmp/ipykernel_4956/1892061918.py:2: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Importing file: mobi_csv/March_2019.csv
Importing file: mobi_csv/November_2018.csv
Failed to import file: mobi_csv/November_2018.csv
Importing file: mobi_csv/January_2024.csv
Importing file: mobi_csv/May_2018.csv
Importing file: mobi_csv/January_2018.csv
Importing file: mobi_csv/July_2022.csv
Importing file: mobi_csv/June_2024.csv
Importing file: mobi_csv/February_2025.csv
Importing file: mobi_csv/January_2022.csv
Importing file: mobi_csv/December_2018.csv
Importing file: mobi_csv/May_2020.csv
Importing file: mobi_csv/December_2024.csv
Importing file: mobi_csv/November_2020.csv
Importing file: mobi_csv/September_2019.csv
Importing file: mobi_csv/December_2021.csv
Importing file: mobi_csv/June_2021.csv
Importing file: mobi_csv/September_2022.csv
Importing file: mobi_csv/September_2020.csv
Importing file: mobi_csv/June_2025.csv
Importing file: mobi_csv/September_2021.csv
Importing file: mobi_csv/June_2020.csv
Importing file: mobi_csv/March_2025.csv
Importing file: mobi_csv/March_2022.csv

/tmp/ipykernel_4956/1892061918.py:2: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Importing file: mobi_csv/October_2020.csv
Importing file: mobi_csv/October_2023.csv
Importing file: mobi_csv/December_2022.csv


...

Importing file: mobi_csv/April_2018.csv
Importing file: mobi_csv/October_2019.csv
Importing file: mobi_csv/September_2023.csv
Importing file: mobi_csv/March_2020.csv
Importing file: mobi_csv/May_2024.csv
Importing file: mobi_csv/May_2021.csv
Importing file: mobi_csv/February_2022.csv
Importing file: mobi_csv/July_2020.csv
Importing file: mobi_csv/September_2025.csv
Importing file: mobi_csv/May_2019.csv
Importing file: mobi_csv/August_2022.csv
Importing file: mobi_csv/March_2018.csv
Importing file: mobi_csv/February_2019.csv
Importing file: mobi_csv/December_2020.csv
Importing file: mobi_csv/May_2025.csv


/tmp/ipykernel_4956/1892061918.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Importing file: mobi_csv/ALL_of_2017.csv
Failed to import file: mobi_csv/ALL_of_2017.csv
Importing file: mobi_csv/August_2019.csv
Importing file: mobi_csv/January_2020.csv
Importing file: mobi_csv/April_2020.csv
Importing file: mobi_csv/December_2023.csv
Importing file: mobi_csv/May_2023.csv
Importing file: mobi_csv/April_2021.csv
Importing file: mobi_csv/April_2025.csv
Importing file: mobi_csv/August_2021.csv
Importing file: mobi_csv/July_2024.csv
Importing file: mobi_csv/March_2023.csv


/tmp/ipykernel_4956/1892061918.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Importing file: mobi_csv/March_2021.csv
Importing file: mobi_csv/November_2024.csv
Importing file: mobi_csv/September_2024.csv
Importing file: mobi_csv/August_2025.csv
Importing file: mobi_csv/February_2021.csv
Importing file: mobi_csv/October_2018.csv
Failed to import file: mobi_csv/October_2018.csv
Importing file: mobi_csv/November_2022.csv
Importing file: mobi_csv/November_2023.csv
Importing file: mobi_csv/April_2019.csv
Importing file: mobi_csv/July_2021.csv
Importing file: mobi_csv/February_2018.csv
Importing file: mobi_csv/Novemeber_2021.csv


In [73]:
query = f"SELECT COUNT(*) FROM {table_name} GROUP BY bike"
test = pd.read_sql_query(query, conn)
test

,COUNT(*)
0,1954966
1,172
2,1823
3,1930
4,347
...,...
2680,1678
2681,1654
2682,31
2683,1
